In [2]:
import numpy as np
import pandas as pd
import sklearn
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from Data_cleaning import *

Chúng ta sẽ lấy các dữ liệu về books, users, rating từ tập dự liệu Book-Crossing.


In [3]:
#import books, users, ratings clean data
books, users, ratings = get_clean_data(path="/content/drive/My Drive/Colab Notebooks/recommendation_system/data/")

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:28

Đây là 5 record đầu về dữ liệu của các quyển sách.


In [4]:
books.head()

,isbn,title,author,pub_year,publisher,url_s,url_m,url_l
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


Đây là dữ liệu user đánh giá các quyển sách.

In [5]:
ratings.head()

,user,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


Chúng ta sẽ đếm số rating mà mỗi quyển sách đã nhận được.

In [6]:
#number of ratings received by each book
usersPerIsbn = ratings['isbn'].value_counts()
usersPerIsbn

0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
              ... 
0441471854       1
0140157425       1
8400000099       1
0451523466       1
0380760436       1
Name: isbn, Length: 340556, dtype: int64

Chúng ta sẽ loại bỏ những quyển sách có lượt rating ít hơn 10 ra khỏi dữ liệu.

In [7]:
books_10 = books[books['isbn'].isin(usersPerIsbn[usersPerIsbn>10].index)]

Sử dụng TF-IDF cho trường tiêu đề của quyển sách.


In [10]:
vectorizer = TfidfVectorizer(analyzer='word')
#build book-title tfidf matrix
tfidf_matrix = vectorizer.fit_transform(books_10['title'])
tfidf_feature_name = vectorizer.get_feature_names()
tfidf_matrix.shape

Sử dụng công thức cosin để tính khoảng cách giữa các item 


In [14]:
# comping cosine similarity matrix using linear_kernal of sklearn
cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

In [15]:
books_10 = books_10.reset_index(drop=True)

In [16]:
indices = pd.Series(books_10['title'].index)

In [17]:
#Function to get the most similar books
def recommend(index, method):
    id = indices[index]
    # Get the pairwise similarity scores of all books compared that book,
    # sorting them and getting top 5
    similarity_scores = list(enumerate(method[id]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:6]
    
    #Get the books index
    books_index = [i[0] for i in similarity_scores]
    
    #Return the top 5 most similar books using integar-location based indexing (iloc)
    return books_10['title'].iloc[books_index]

In [18]:
#input the index of the book
recommend(1000, cosine_similarity)

11488                                          The Odyssey
14633                                          The Odyssey
4000                                           Killer Hair
9422                             The Odyssey (Classics S.)
4174     Longitude: The True Story of a Lone Genius Who...
Name: title, dtype: object

In [19]:
books_10.iloc[1000]

isbn                                                076790351X
title        Beethoven's Hair : An Extraordinary Historical...
author                                          RUSSELL MARTIN
pub_year                                                  2001
publisher                                             Broadway
url_s        http://images.amazon.com/images/P/076790351X.0...
url_m        http://images.amazon.com/images/P/076790351X.0...
url_l        http://images.amazon.com/images/P/076790351X.0...
Name: 1000, dtype: object